In [9]:
import os
import csv
import requests
import boto3
from io import StringIO
from datetime import datetime, timedelta
import json
import pytz
import concurrent.futures
from threading import Lock
import time

# Add dotenv support for local development
try:
    from dotenv import load_dotenv
    load_dotenv()  # This will load your .env file from the same directory
    print("✅ .env file loaded successfully")
except ImportError:
    print("⚠️  python-dotenv not installed - using system environment variables")
except Exception as e:
    print(f"⚠️  Could not load .env file: {e}")

# Thread-safe counters
progress_lock = Lock()
processed_count = 0
api_call_count = 0

# Test event for when no event is provided (Jupyter/local testing)
TEST_EVENT = {
    "environment": "test",
    "max_workers": 10,
    "ticker_limit": 100,
    "page_limit": 5
}

def make_polygon_request(url, headers, symbol=None, max_retries=3):
    """Make a Polygon API request with retry logic"""
    global api_call_count
    
    for attempt in range(max_retries):
        try:
            with progress_lock:
                api_call_count += 1
                
            response = requests.get(url, headers=headers, timeout=15)
            
            if response.status_code == 200:
                data = response.json()
                if data.get('status') == 'OK':
                    return data
                else:
                    print(f"API returned non-OK status for {symbol}: {data.get('status')}")
                    return None
            elif response.status_code == 429:  # Rate limited
                wait_time = 2 ** attempt
                print(f"Rate limited on {symbol}, waiting {wait_time}s...")
                time.sleep(wait_time)
                continue
            else:
                print(f"HTTP {response.status_code} for {symbol}: {response.text[:200]}")
                return None
                
        except Exception as e:
            if attempt == max_retries - 1:
                print(f"Final attempt failed for {symbol}: {e}")
                return None
            else:
                time.sleep(1)
                continue
    
    return None

def get_all_us_stocks(api_key, page_limit=None, per_page_limit=1000):
    """Get all US stocks from Polygon tickers endpoint"""
    headers = {"Authorization": f"Bearer {api_key}"}
    all_tickers = []
    next_url = f"https://api.polygon.io/v3/reference/tickers?market=stocks&locale=us&active=true&limit={per_page_limit}"
    
    page_count = 0
    max_pages = page_limit or 100  # Default safety limit, but configurable
    while next_url and page_count < max_pages:
        page_count += 1
        print(f"Fetching page {page_count} of tickers...")
        
        data = make_polygon_request(next_url, headers, "tickers_page")
        if not data or 'results' not in data:
            break
            
        tickers = data['results']
        all_tickers.extend([ticker['ticker'] for ticker in tickers])
        
        # Get next page URL
        next_url = data.get('next_url')
        if next_url:
            # Check if next_url is already a full URL or just a path
            if next_url.startswith('https://'):
                # It's already a full URL, just add the API key
                if '?' in next_url:
                    next_url = f"{next_url}&apikey={api_key}"
                else:
                    next_url = f"{next_url}?apikey={api_key}"
            else:
                # It's just a path, prepend the domain
                if '?' in next_url:
                    next_url = f"https://api.polygon.io{next_url}&apikey={api_key}"
                else:
                    next_url = f"https://api.polygon.io{next_url}?apikey={api_key}"
        
        print(f"Page {page_count}: Found {len(tickers)} tickers, total: {len(all_tickers)}")
        
        # Small delay between pages
        time.sleep(0.1)
    
    if page_count >= max_pages:
        print(f"Reached page limit of {max_pages}. Total tickers collected: {len(all_tickers)}")
    else:
        print(f"Completed all pages. Total US stocks found: {len(all_tickers)}")
    return all_tickers

def get_essential_stock_data(symbol, api_key):
    """Get essential trading data with minimal API calls (2 calls per ticker)"""
    headers = {"Authorization": f"Bearer {api_key}"}
    stock_data = {'ticker': symbol}
    
    # Get current time for column naming
    cst = pytz.timezone('America/Chicago')
    now_cst = datetime.now(cst)
    time_str = now_cst.strftime('%H:%M')
    
    # 1. Company Details (for name and market cap)
    company_url = f"https://api.polygon.io/v3/reference/tickers/{symbol}"
    company_data = make_polygon_request(company_url, headers, symbol)
    if company_data and 'results' in company_data:
        result = company_data['results']
        stock_data.update({
            'name': result.get('name', 'N/A'),
            'market_cap_millions': result.get('market_cap', 0) / 1_000_000 if result.get('market_cap') else 0,
            'primary_exchange': result.get('primary_exchange', 'N/A')
        })
    else:
        stock_data.update({
            'name': 'N/A',
            'market_cap_millions': 0,
            'primary_exchange': 'N/A'
        })
    
    # 2. Previous Day Aggregates (OHLCV)
    prev_day_url = f"https://api.polygon.io/v2/aggs/ticker/{symbol}/prev"
    prev_data = make_polygon_request(prev_day_url, headers, symbol)
    if prev_data and 'results' in prev_data and len(prev_data['results']) > 0:
        result = prev_data['results'][0]
        
        # Extract OHLCV data
        open_price = result.get('o')
        high_price = result.get('h')
        low_price = result.get('l')
        close_price = result.get('c')  # This is previous close
        volume = result.get('v')
        
        stock_data.update({
            'open': f"{open_price:.2f}" if open_price is not None else 'N/A',
            'high': f"{high_price:.2f}" if high_price is not None else 'N/A',
            'low': f"{low_price:.2f}" if low_price is not None else 'N/A',
            'previous_close': f"{close_price:.2f}" if close_price is not None else 'N/A',
            'volume': f"{int(volume)}" if volume is not None else 'N/A'
        })
        
        # Current price with timestamp (using previous close as proxy)
        current_price_col = f'current_price_{time_str}'
        stock_data[current_price_col] = stock_data['previous_close']
        
        # Calculate open_pct_chg (current price vs open price)
        open_pct_chg_col = f'open_pct_chg_{time_str}'
        if open_price is not None and close_price is not None and open_price != 0:
            pct_change = ((close_price - open_price) / open_price) * 100
            stock_data[open_pct_chg_col] = f"{pct_change:+.2f}%"
        else:
            stock_data[open_pct_chg_col] = 'N/A'
        
        # Store raw values for potential previous_pct_chg calculation
        stock_data['_raw_current_price'] = close_price if close_price is not None else None
        stock_data['_current_time'] = time_str
        
    else:
        stock_data.update({
            'open': 'N/A',
            'high': 'N/A', 
            'low': 'N/A',
            'previous_close': 'N/A',
            'volume': 'N/A'
        })
        
        # Still need the timestamped columns even if no data
        current_price_col = f'current_price_{time_str}'
        open_pct_chg_col = f'open_pct_chg_{time_str}'
        stock_data[current_price_col] = 'N/A'
        stock_data[open_pct_chg_col] = 'N/A'
        stock_data['_raw_current_price'] = None
        stock_data['_current_time'] = time_str
    
    # Format market cap
    if stock_data['market_cap_millions'] > 0:
        stock_data['market_cap_millions'] = f"{stock_data['market_cap_millions']:.2f}"
    else:
        stock_data['market_cap_millions'] = 'N/A'
    
    return stock_data

def process_single_ticker(symbol, api_key):
    """Process a single ticker with minimal API calls"""
    global processed_count
    
    try:
        stock_data = get_essential_stock_data(symbol, api_key)
        
        # Update progress counter
        with progress_lock:
            processed_count += 1
            if processed_count % 100 == 0:
                print(f"Processed {processed_count} tickers... (API calls: {api_call_count})")
        
        return stock_data, None
        
    except Exception as e:
        error_msg = f"Error processing {symbol}: {str(e)}"
        print(error_msg)
        return None, error_msg

def lambda_handler(event=None, context=None):
    global processed_count, api_call_count
    processed_count = 0
    api_call_count = 0
    
    # Auto-detect mode: if no event provided, assume Jupyter/local mode
    jupyter_mode = event is None
    
    if jupyter_mode:
        event = TEST_EVENT
        print("🧪 No event provided - Running in JUPYTER/LOCAL mode with test event")
        print(f"Test event: {event}")
    else:
        print("🚀 Event provided - Running in LAMBDA mode")
    
    # Initialize AWS clients (skip if in Jupyter mode and no AWS setup)
    try:
        s3 = boto3.client('s3')
        sns = boto3.client('sns')
        aws_available = True
        if jupyter_mode:
            print("✅ AWS clients initialized successfully")
    except Exception as e:
        if jupyter_mode:
            print(f"⚠️  AWS clients not available (expected in Jupyter): {e}")
            s3 = None
            sns = None
            aws_available = False
        else:
            raise e
    
    # Get environment variables
    bucket_name = os.getenv('BUCKET_NAME')
    polygon_api_key = os.getenv('POLYGON_API_KEY')
    sns_topic_arn = os.getenv('SNS_TOPIC_ARN')
    
    if not polygon_api_key:
        raise ValueError("POLYGON_API_KEY environment variable is required")
    
    # Configuration
    is_test = event.get('environment') == 'test'
    max_workers = event.get('max_workers', 20)
    ticker_limit = event.get('ticker_limit', 5000 if is_test else None)
    page_limit = event.get('page_limit', None)
    
    # Generate file paths
    input_date = datetime.now().strftime('%Y%m%d')
    timestamp = datetime.now().strftime('%H%M')
    
    if jupyter_mode:
        # Jupyter mode: Use simple daily filename for appending
        output_file_name = f'stock_data_{input_date}.csv'
        output_file_key = output_file_name
    else:
        # Lambda mode: Use S3 path structure
        output_file_key = f'stock_data/{input_date}/stock_data_{input_date}.csv'
    
    # Check if we're appending to existing data
    existing_data = {}
    
    if jupyter_mode:
        # Try to read existing local file
        try:
            with open(output_file_name, 'r', encoding='utf-8') as f:
                existing_content = f.read()
                existing_reader = csv.DictReader(StringIO(existing_content))
                for row in existing_reader:
                    existing_data[row['ticker']] = row
            print(f"📂 Found existing local file with {len(existing_data)} tickers")
        except FileNotFoundError:
            print(f"📂 No existing local file found, creating new: {output_file_name}")
            existing_data = {}
        except Exception as e:
            print(f"📂 Error reading existing file: {e}")
            existing_data = {}
    elif aws_available:
        # Try to read existing S3 file
        try:
            existing_obj = s3.get_object(Bucket=bucket_name, Key=output_file_key)
            existing_content = existing_obj['Body'].read().decode('utf-8')
            existing_reader = csv.DictReader(StringIO(existing_content))
            for row in existing_reader:
                existing_data[row['ticker']] = row
            print(f"📂 Found existing S3 file with {len(existing_data)} tickers")
        except Exception as e:
            print(f"📂 No existing S3 file found, creating new: {e}")
            existing_data = {}
    
    try:
        print("Starting essential stock data collection...")
        start_time = time.time()
        
        # Get all US stock tickers
        if is_test:
            tickers = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA'][:ticker_limit] if ticker_limit else ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA']
            print(f"Test mode: Processing {len(tickers)} sample tickers")
        else:
            print("Fetching all US stock tickers...")
            all_tickers = get_all_us_stocks(polygon_api_key, page_limit=page_limit)
            tickers = all_tickers[:ticker_limit] if ticker_limit else all_tickers
            print(f"Production mode: Processing {len(tickers)} tickers")
        
        # Process all tickers in parallel
        valid_data = []
        errors = []
        
        print(f"Starting parallel processing with {max_workers} workers...")
        print(f"Making 2 API calls per ticker (company info + OHLCV data)")
        
        with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
            # Submit all tasks
            future_to_ticker = {
                executor.submit(process_single_ticker, ticker, polygon_api_key): ticker
                for ticker in tickers
            }
            
            # Collect results as they complete
            for future in concurrent.futures.as_completed(future_to_ticker):
                ticker = future_to_ticker[future]
                try:
                    stock_data, error = future.result()
                    if stock_data:
                        # Calculate previous_pct_chg if we have existing data
                        if ticker in existing_data:
                            current_time = stock_data['_current_time']
                            current_price = stock_data['_raw_current_price']
                            
                            # Find the most recent previous current_price column
                            prev_price_col = None
                            prev_price_value = None
                            
                            for col, value in existing_data[ticker].items():
                                if col.startswith('current_price_') and col != f'current_price_{current_time}':
                                    if value != 'N/A':
                                        try:
                                            prev_price_value = float(value)
                                            prev_price_col = col
                                        except:
                                            continue
                            
                            # Calculate previous_pct_chg
                            prev_pct_chg_col = f'previous_pct_chg_{current_time}'
                            if prev_price_value is not None and current_price is not None and prev_price_value != 0:
                                pct_change = ((current_price - prev_price_value) / prev_price_value) * 100
                                stock_data[prev_pct_chg_col] = f"{pct_change:+.2f}%"
                            else:
                                stock_data[prev_pct_chg_col] = 'N/A'
                            
                            # Merge with existing data (preserve old columns, add new ones)
                            merged_data = existing_data[ticker].copy()
                            merged_data.update(stock_data)
                            stock_data = merged_data
                        else:
                            # New ticker, add empty previous_pct_chg
                            current_time = stock_data['_current_time']
                            prev_pct_chg_col = f'previous_pct_chg_{current_time}'
                            stock_data[prev_pct_chg_col] = 'N/A'
                        
                        # Remove internal helper fields
                        stock_data.pop('_raw_current_price', None)
                        stock_data.pop('_current_time', None)
                        
                        valid_data.append(stock_data)
                    if error:
                        errors.append(error)
                except Exception as e:
                    error_msg = f"Exception processing {ticker}: {e}"
                    print(error_msg)
                    errors.append(error_msg)
        
        processing_time = time.time() - start_time
        print(f"Processing completed in {processing_time:.1f} seconds")
        print(f"Valid records: {len(valid_data)}, Errors: {len(errors)}, API calls: {api_call_count}")
        
        if not valid_data:
            raise Exception("No valid data collected")
        
        # Get the current time for the dynamic column names
        cst = pytz.timezone('America/Chicago')
        now_cst = datetime.now(cst)
        time_str = now_cst.strftime('%H:%M')
        current_price_col = f'current_price_{time_str}'
        open_pct_chg_col = f'open_pct_chg_{time_str}'
        prev_pct_chg_col = f'previous_pct_chg_{time_str}'
        
        # Collect all possible columns from the data
        all_columns = set()
        for row in valid_data:
            all_columns.update(row.keys())
        
        # Define base column order
        base_columns = [
            'ticker',
            'name', 
            'market_cap_millions',
            'volume',
            'previous_close',
            'open',
            'high',
            'low'
        ]
        
        # Separate timestamped columns and sort them chronologically
        current_price_cols = sorted([col for col in all_columns if col.startswith('current_price_')])
        open_pct_chg_cols = sorted([col for col in all_columns if col.startswith('open_pct_chg_')])
        prev_pct_chg_cols = sorted([col for col in all_columns if col.startswith('previous_pct_chg_')])
        
        # Build final column order: base columns + timestamped columns in chronological order
        fieldnames = base_columns.copy()
        
        # Add timestamped columns in sets, chronologically
        all_timestamps = set()
        for col in current_price_cols + open_pct_chg_cols + prev_pct_chg_cols:
            timestamp = col.split('_')[-1]  # Extract HH:MM
            all_timestamps.add(timestamp)
        
        # Sort timestamps chronologically
        sorted_timestamps = sorted(all_timestamps)
        
        # Add columns for each timestamp in order: current_price, open_pct_chg, previous_pct_chg
        for timestamp in sorted_timestamps:
            for col_type in ['current_price', 'open_pct_chg', 'previous_pct_chg']:
                col_name = f'{col_type}_{timestamp}'
                if col_name in all_columns:
                    fieldnames.append(col_name)
        
        # Create CSV output
        output = StringIO()
        writer = csv.DictWriter(output, fieldnames=fieldnames, extrasaction='ignore')
        writer.writeheader()
        
        # Sort by market cap (descending) then by ticker
        def sort_key(item):
            try:
                market_cap = float(item.get('market_cap_millions', 0)) if item.get('market_cap_millions') != 'N/A' else 0
                return (-market_cap, item.get('ticker', ''))
            except:
                return (0, item.get('ticker', ''))
        
        valid_data.sort(key=sort_key)
        
        for row in valid_data:
            writer.writerow(row)
        
        # Handle output based on mode
        if jupyter_mode:
            # Save locally in Jupyter mode with append functionality
            with open(output_file_name, 'w', newline='', encoding='utf-8') as f:
                f.write(output.getvalue())
            print(f"📁 Data {'appended to' if existing_data else 'saved as'} local file: {output_file_name}")
            file_location = output_file_name
        elif aws_available:
            # Upload to S3 in Lambda mode
            print(f"Uploading data to S3: {output_file_key}")
            s3.put_object(
                Bucket=bucket_name,
                Key=output_file_key,
                Body=output.getvalue(),
                ContentType='text/csv'
            )
            file_location = output_file_key
        else:
            # Fallback: save locally even in Lambda mode if AWS not available
            fallback_filename = f'stock_data_{input_date}_{timestamp}.csv'
            with open(fallback_filename, 'w', newline='', encoding='utf-8') as f:
                f.write(output.getvalue())
            print(f"📁 AWS not available, data saved locally as: {fallback_filename}")
            file_location = fallback_filename
        
        # Generate presigned URL (only if AWS available)
        presigned_url = None
        if aws_available and s3:
            try:
                presigned_url = s3.generate_presigned_url(
                    'get_object',
                    Params={'Bucket': bucket_name, 'Key': output_file_key},
                    ExpiresIn=604800  # 7 days
                )
            except Exception as e:
                print(f"Could not generate presigned URL: {e}")
        
        # Calculate statistics
        success_rate = (len(valid_data) / len(tickers)) * 100 if tickers else 0
        avg_api_calls_per_ticker = api_call_count / len(tickers) if tickers else 0
        
        # Send notification (only if AWS available)
        if aws_available and sns:
            message = (
                f'📊 ESSENTIAL STOCK DATA COLLECTION COMPLETE\n\n'
                f'🎯 STATISTICS:\n'
                f'• Environment: {"TEST" if is_test else "PRODUCTION"}\n'
                f'• Total tickers processed: {len(tickers):,}\n'
                f'• Successful records: {len(valid_data):,}\n'
                f'• Failed records: {len(errors):,}\n'
                f'• Success rate: {success_rate:.1f}%\n'
                f'• Processing time: {processing_time:.1f} seconds\n'
                f'• API calls made: {api_call_count:,} (2 per ticker)\n'
                f'• Avg API calls per ticker: {avg_api_calls_per_ticker:.1f}\n\n'
                f'📈 DATA INCLUDES:\n'
                f'• Ticker symbol\n'
                f'• Company name\n'
                f'• Market cap (millions)\n'
                f'• Primary exchange\n'
                f'• OHLCV data (Open, High, Low, Close, Volume)\n'
                f'• Current price\n\n'
                f'💾 Download link (expires in 7 days):\n{presigned_url or "N/A"}'
            )
            
            if errors and len(errors) <= 10:
                message += f'\n\n❌ ERRORS:\n' + '\n'.join(errors[:10])
            elif errors:
                message += f'\n\n❌ First 10 errors:\n' + '\n'.join(errors[:10])
            
            try:
                sns.publish(
                    TopicArn=sns_topic_arn,
                    Subject=f'📊 Essential Stock Data Complete - {len(valid_data):,} stocks',
                    Message=message
                )
            except Exception as e:
                print(f"Could not send SNS notification: {e}")
        
        # Print summary for Jupyter mode
        if jupyter_mode:
            print("\n" + "="*60)
            print("🎉 DATA COLLECTION COMPLETE!")
            print("="*60)
            print(f"📊 Processed {len(tickers):,} tickers")
            print(f"✅ Successful: {len(valid_data):,} ({success_rate:.1f}%)")
            print(f"❌ Errors: {len(errors):,}")
            print(f"⏱️  Time: {processing_time:.1f}s")
            print(f"🔌 API calls: {api_call_count:,} (2 per ticker)")
            print(f"📁 File: {file_location}")
            print(f"🔄 Mode: {'Appended to existing' if existing_data else 'Created new'} daily file")
            if errors:
                print(f"\n🚨 Sample errors: {errors[:3]}")
        
        return {
            'statusCode': 200,
            'body': {
                'message': 'Successfully collected essential stock data',
                'environment': 'test' if is_test else 'production',
                'mode': 'jupyter' if jupyter_mode else 'lambda',
                'append_mode': len(existing_data) > 0,
                'existing_tickers': len(existing_data),
                'total_tickers': len(tickers),
                'successful_records': len(valid_data),
                'errors': len(errors),
                'success_rate': f"{success_rate:.1f}%",
                'processing_time_seconds': round(processing_time, 1),
                'api_calls_made': api_call_count,
                'output_file': file_location,
                'download_url': presigned_url
            }
        }
        
    except Exception as e:
        error_message = f'Error collecting essential stock data: {str(e)}'
        print(f"Exception: {error_message}")
        
        # Send error notification (only if AWS available)
        if aws_available and sns:
            try:
                sns.publish(
                    TopicArn=sns_topic_arn,
                    Subject='❌ Essential Stock Data Collection Failed',
                    Message=f'Error: {error_message}\n\nProcessed {processed_count} tickers before failure.\nAPI calls made: {api_call_count}'
                )
            except Exception as sns_error:
                print(f"Could not send error notification: {sns_error}")
        
        return {
            'statusCode': 500,
            'body': {'error': error_message, 'processed_count': processed_count, 'api_calls': api_call_count}
        }
result = lambda_handler()
# For Lambda: gets called automatically with event

✅ .env file loaded successfully
🧪 No event provided - Running in JUPYTER/LOCAL mode with test event
Test event: {'environment': 'test', 'max_workers': 10, 'ticker_limit': 100, 'page_limit': 5}
✅ AWS clients initialized successfully
📂 Found existing local file with 5 tickers
Starting essential stock data collection...
Test mode: Processing 5 sample tickers
Starting parallel processing with 10 workers...
Making 2 API calls per ticker (company info + OHLCV data)
Processing completed in 0.4 seconds
Valid records: 5, Errors: 0, API calls: 10
📁 Data appended to local file: stock_data_20250601.csv

🎉 DATA COLLECTION COMPLETE!
📊 Processed 5 tickers
✅ Successful: 5 (100.0%)
❌ Errors: 0
⏱️  Time: 0.4s
🔌 API calls: 10 (2 per ticker)
📁 File: stock_data_20250601.csv
🔄 Mode: Appended to existing daily file
